# Last Cycle
Now that the cycle stabilized we can cycle one more time to track relevant quantities:
- Mutual Information
- Work
- Heat
- Internal Energy (?)

In [1]:
using Revise
using Dates

In [ ]:
dir = "cascade_25D_I1";

includet("./modules/OpticalCavity.jl")
includet("./modules/Thermodynamics.jl")
includet("./modules/Phaseonium.jl")
includet("./modules/BosonicOperators.jl")
includet("./modules/Measurements.jl")

includet("./src/main.jl")
config, ndims = init(dir)

total_cycles = config["loading"]["past_cycles"]
system_evolution = []

const PLOT_EVO = false;

# Load or create state
state = load_or_create(dir, config)
# Reset state
state.ρ₁_evolution = [] 
state.ρ₂_evolution = []
state.c₁_evolution = []
state.c₂_evolution = []

# Report file
open(dir * "/report.txt", "w") do io
    println(io, "Starting simulation at time $(now()) with config:")
    print_config(io, config)
end
# Visualization folder
mkpath(dir * "/visualization")

check_cutoff(state.ρ, ndims)

Simulation 5 - 25D - Fast thermalization - High entanglement: what happens to the efficiency?

Initial Temperature of the Cavity: 2.25
Apparent Temperature carried by Hot Phaseonium atoms: 20.0
Apparent Temperature carried by Cold Phaseonium atoms: 0.005
Starting with a new cascade system (contracted)
Initial Temperature of the Cavities: 2.25 - 2.8499999999999996


LoadError: UndefVarError: `THERMALIZED` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
# Cycle

isochore_time = config["stroke_time"]["isochore"]
isochore_samplings = config["samplings"]["isochore"]
adiabatic_time = config["stroke_time"]["adiabatic"]
adiabatic_samplings = config["samplings"]["adiabatic"]

total_cycle_time = isochore_samplings * 2 + 2 * adiabatic_samplings
cycle_steps = []  # This keeps track of the total time of each stroke

open(dir * "/report.txt", "a") do io
    redirect_stdout(io)

    global state
    global system_evolution
    global total_cycles

    try
        for t in 1:config["cycles"]
            println("Cycle $t")
            state, system_evolution = Main.cycle(
                state, Δt, system_evolution, cycle_steps, 
                isochore_time, isochore_samplings, adiabatic_time, adiabatic_samplings);
            total_cycles += 1
            flush(io)

            if PLOT_EVO
                sys = state.ρ₁_evolution
                cav = state.c₁_evolution
                g1 = measure_and_plot("Frequency", "Energy", sys, cav, "Cycle $t S1"; α=π)
                savefig(g1, dir * "/visualization/S1$(total_cycles)C.png")
                sys = state.ρ₂_evolution
                cav = state.c₂_evolution
                g2 = measure_and_plot("Frequency", "Energy", sys, cav, "Cycle $t S2"; α=π)
                savefig(g2, dir * "/visualization/S2$(total_cycles)C.png")
            end
        end
    catch e 
        println("\n\nError '$e' at time $(now()) after $(length(system_evolution)) steps")
        error("Error $e: ending cycle")
    else
        println("\n\nFinished at time $(now()) after $(length(system_evolution)) steps")
        println("**********************************")
        serialize(dir * "/state_$(total_cycles)C.jl", state)
        serialize(dir * "/state_evolution_$(total_cycles)C.jl", system_evolution)
        serialize(dir * "/cavity1_evolution_$(total_cycles)C.jl", state.c₁_evolution)
        serialize(dir * "/cavity2_evolution_$(total_cycles)C.jl", state.c₂_evolution)
    end
end
println("Cycle len: $(size(system_evolution)[1] / total_cycles) steps")

### Check stability of the Cycle

In [ ]:
println(size(system_evolution))
println(size(state.c₁_evolution))
state_evolution1 = state.ρ₁_evolution[end-total_cycle_time+1:end] # last cycle
cavity_evolution1 = state.c₁_evolution[end-total_cycle_time+1:end] # last cycle
state_evolution2 = state.ρ₂_evolution[end-total_cycle_time+1:end] # last cycle
cavity_evolution2 = state.c₂_evolution[end-total_cycle_time+1:end] # last cycle

g = measure_and_plot("Frequency", "Energy", state_evolution1, cavity_evolution1, "S1"; α=π, title="Cavities last cycle")
g = measure_and_plot("Frequency", "Energy", state_evolution2, cavity_evolution2, "S2"; α=π, g=g)

In [ ]:
savefig(g, dir * "/last_cycle.png")

## Save Last Cycle

In [ ]:
α0 = config["cavity1"]["alpha"]

last_evolution = system_evolution[end-total_cycle_time+1:end]
ω₁_evolution = [α0 / l1 for l1 in state.c₁_evolution[end-total_cycle_time+1:end]]
ω₂_evolution = [α0 / l2 for l2 in state.c₂_evolution[end-total_cycle_time+1:end]]

dict_evolution = Dict(string(i)=>last_evolution[i] for i in 1:length(last_evolution))
cycle_steps = Vector{Float64}(cycle_steps)

npzwrite(dir * "/cycle_steps.npy", cycle_steps[end-3:end])
npzwrite(dir * "/cavities_evolution_1cycle.npz", c1=ω₁_evolution, c2=ω₂_evolution)
npzwrite(dir * "/cascade_evolution_1cycle.npz", dict_evolution)